In [ ]:
import json
import numpy as np
import os
import pandas as pd

from tqdm import tqdm

In [ ]:
# Load preprocessed patient data
df_patientdata = pd.read_csv('data/preprocessed/patient_data.csv', dtype={'stay_id': str, 'subject_id': str, 'acuity': str, 'disposition': str, 'complexity': str, 'los': float})

df_patientdata

In [ ]:
# Load preprocessed event logs
df_eventlogs = pd.read_csv('data/preprocessed/event_logs.csv', dtype={'case_id': str, 'activity_name': str})
df_eventlogs['timestamp'] = pd.to_datetime(df_eventlogs['timestamp'])

df_eventlogs

In [ ]:
# Get list of ED activities
list_activities = sorted(set(df_eventlogs['activity_name']))

list_activities

In [ ]:
# Group event logs per case ID
df_eventlogs_per_caseid = df_eventlogs.groupby(['case_id']).aggregate({'activity_name': list, 'timestamp': list}).reset_index()

df_eventlogs_per_caseid

In [ ]:
# Merge patient data with event logs
df_ehrs = pd.merge(df_patientdata, df_eventlogs_per_caseid, how='right', left_on='stay_id', right_on='case_id')
df_ehrs.dropna(inplace=True)

df_ehrs

In [ ]:
# Impose a 5-minute temporal resolution
TEMPORAL_RESOLUTION = 5  # in minutes
new_activity_col, new_timestamp_col = [], []
for idx in tqdm(df_ehrs.index):
    row = df_ehrs.loc[idx]

    activity_list, timestamp_list = row['activity_name'], row['timestamp']

    new_activity_list, new_timestamp_list = [activity_list[0]], [timestamp_list[0]]
    for act_idx, time_idx in zip(activity_list[1:], timestamp_list[1:]):
        if act_idx != new_activity_list[-1]:
            new_timestamp_list.append(time_idx)
            new_activity_list.append(act_idx)
        else:
            if (time_idx - new_timestamp_list[-1]).total_seconds() > (60 * TEMPORAL_RESOLUTION):
                new_timestamp_list.append(time_idx)
                new_activity_list.append(act_idx)

    new_timestamp_col.append(new_timestamp_list)
    new_activity_col.append(''.join(new_activity_list))

df_ehrs.loc[:, 'activity_name'] = new_activity_col
df_ehrs.loc[:, 'timestamp'] = np.array(new_timestamp_col, dtype=object)

df_ehrs

In [ ]:
# Add column for retrospective cohort groupname
df_ehrs.loc[:, 'groupname'] = df_ehrs.loc[:, 'acuity'] + '-' + df_ehrs.loc[:, 'disposition'] + '-' + df_ehrs.loc[:, 'complexity']

# Add column for case length
df_ehrs.loc[:, 'case_len'] = df_ehrs.activity_name.str.len()

df_ehrs

In [ ]:
df_ehrs = df_ehrs[df_ehrs['activity_name'].str.startswith('A')].copy()

df_ehrs

In [ ]:
df_ehrs = df_ehrs[df_ehrs['activity_name'].str.endswith('G')].copy()

df_ehrs

In [ ]:
df_ehrs = df_ehrs[df_ehrs['case_len'] >= 3].copy()

df_ehrs

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot percentage of EHRs with the activity
x_list, y_list = [], []
for act_name in sorted(list_activities):
    df_act = df_ehrs[df_ehrs['activity_name'].str.contains(act_name)]
    print(act_name, len(df_act)/len(df_ehrs) * 100)
    x_list.append(act_name)
    y_list.append(round(len(df_act)/len(df_ehrs) * 100, 2))
ax = sns.barplot(x=x_list, y=y_list, color='blue')
ax.bar_label(ax.containers[0])
plt.show()

In [ ]:
# Save case length distribution targets
outpath_caselen = 'params/caselen'
if not os.path.exists(outpath_caselen):
    os.makedirs(outpath_caselen)

# For entire patient population
with open(f'{outpath_caselen}/caselen_overall.txt', 'w+') as filehandle:
    json.dump(df_ehrs.case_len.to_list(), filehandle)

# For each patient indicator
for variable_name in ['acuity', 'disposition', 'complexity']:
    for variable_val in set(df_ehrs[variable_name].values):
        df_patientgroup = df_ehrs[df_ehrs[variable_name] == variable_val]

        patientgroup_caselen = df_patientgroup.case_len.to_list()
        with open(f'{outpath_caselen}/caselen_{variable_name}_{variable_val}.txt', 'w+') as filehandle:
            json.dump(patientgroup_caselen, filehandle)

# For every combination of all patient indicators
for variable_val in set(df_ehrs['groupname'].values):
    df_patientgroup = df_ehrs[df_ehrs['groupname'] == variable_val]

    patientgroup_caselen = df_patientgroup.case_len.to_list()
    with open(f'{outpath_caselen}/caselen_groupname_{variable_val}.txt', 'w+') as filehandle:
        json.dump(patientgroup_caselen, filehandle)

In [ ]:
# Save length of stay distribution targets
outpath_los = 'params/los'
if not os.path.exists(outpath_los):
    os.makedirs(outpath_los)

# For entire patient population
baseline_los = df_ehrs['los'].to_list()
with open(f'{outpath_los}/los_overall.txt', 'w+') as filehandle:
    json.dump(baseline_los, filehandle)

# For each patient indicator
for variable_name in ['acuity', 'disposition', 'complexity']:
    for variable_val in set(df_ehrs[variable_name].values):
        df_patientgroup = df_ehrs[df_ehrs[variable_name] == variable_val]

        patientgroup_los = df_patientgroup['los'].to_list()
        with open(f'{outpath_los}/los_{variable_name}_{variable_val}.txt', 'w+') as filehandle:
            json.dump(patientgroup_los, filehandle)

# For every combination of all patient indicators
for variable_val in set(df_ehrs['groupname'].values):
    df_patientgroup = df_ehrs[df_ehrs['groupname'] == variable_val]

    patientgroup_los = df_patientgroup['los'].to_list()
    with open(f'{outpath_los}/los_groupname_{variable_val}.txt', 'w+') as filehandle:
        json.dump(patientgroup_los, filehandle)

In [ ]:
# Save frequency distributions of patient indicator
outpath_frequency = 'params/frequency'
if not os.path.exists(outpath_frequency):
    os.makedirs(outpath_frequency)

# For acuity
df_frequency = df_ehrs.groupby(['acuity']).case_id.count() / df_ehrs.case_id.count()
df_frequency = df_frequency.rename('percent')
df_frequency.to_csv(f'{outpath_frequency}/frequency.csv')

# For disposition and complexity per acuity
df_frequency = df_ehrs.groupby(['acuity', 'disposition', 'complexity']).case_id.count() / df_ehrs.groupby('acuity').case_id.count()
df_frequency = df_frequency.rename('percent')
df_frequency.to_csv(f'{outpath_frequency}/frequency_groupname.csv')

In [ ]:
# Save independent branching probabilities for both prospective (acuity) and retrosective (groupname) cohort types
outpath_branching = 'params/branching'
if not os.path.exists(outpath_branching):
    os.makedirs(outpath_branching)

for variable_name in tqdm(['groupname', 'acuity']):
    for variable_val in tqdm(set(df_ehrs[variable_name].values)):
        df_patientgroup = df_ehrs[df_ehrs[variable_name] == variable_val]

        dict_edge_count_ind = {}
        for edge_from in list_activities:
            dict_edge_count_ind[edge_from] = {}
            for edge_to in list_activities:
                dict_edge_count_ind[edge_from][edge_to] = 0

        for case_act in df_patientgroup['activity_name']:
            for idx in range(1,len(case_act)):
                edge_from = case_act[idx-1]
                edge_to = case_act[idx]

                dict_edge_count_ind[edge_from][edge_to] += 1

        df_edge_count_ind = pd.DataFrame.from_dict(dict_edge_count_ind).T
        df_edge_count_ind = df_edge_count_ind.div(df_edge_count_ind.sum(axis=1), axis=0)
        df_edge_count_ind = df_edge_count_ind.fillna(0.0)
        df_edge_count_ind.index.name = 'name'
        df_edge_count_ind.to_csv(f'{outpath_branching}/independent_branching_{variable_name}_{variable_val}.csv')

In [ ]:
# Save conditional branching probabilities for both prospective (acuity) and retrosective (groupname) cohort types
for variable_name in tqdm(['groupname', 'acuity']):
    for variable_val in tqdm(set(df_ehrs[variable_name].values)):
        df_patientgroup = df_ehrs[df_ehrs[variable_name] == variable_val]

        caselen_max = max(set([x for x in df_patientgroup['activity_name'].str.len()]))

        dict_caselen = {}
        for caselen_idx in range(1, caselen_max+1):
            dict_caselen[caselen_idx] = {}

        for case_act in df_patientgroup['activity_name']:
            if case_act not in dict_caselen[len(case_act)]:
                dict_caselen[len(case_act)][case_act] = {'count': 1, 'prob': 0}
            else:
                dict_caselen[len(case_act)][case_act]['count'] += 1

            for idx in range(len(case_act)-1):
                first_part = case_act[:idx+1]
                case_len = len(first_part)

                if first_part not in dict_caselen[case_len]:
                    dict_caselen[case_len][first_part] = {'count': 1, 'prob': 0}
                else:
                    dict_caselen[case_len][first_part]['count'] += 1

        for case_len, possible_cases in sorted(dict_caselen.items()):
            for case_idx in possible_cases.keys():
                if case_len > 1:
                    prior_count = dict_caselen[case_len-1][case_idx[:case_len-1]]['count']
                    prior_prob = dict_caselen[case_len-1][case_idx[:case_len-1]]['prob']
                    possible_cases[case_idx]['prob'] = (possible_cases[case_idx]['count'] / len(df_ehrs)) / (prior_count / len(df_ehrs))
                else:
                    possible_cases[case_idx]['prob'] = possible_cases[case_idx]['count'] / len(df_ehrs)

        dict_edge_count_con = {}
        for caselen_idx in dict_caselen.keys():
            if caselen_idx != 1:
                for case_idx in sorted(dict_caselen[caselen_idx]):
                    prefix_name = case_idx[:-1]
                    final_char = case_idx[-1]
                    if prefix_name not in dict_edge_count_con.keys():
                        dict_edge_count_con[prefix_name] = {value: 0 for value in list_activities}
                    dict_edge_count_con[prefix_name][final_char] = dict_caselen[caselen_idx][case_idx]['prob']

        df_edge_count_con = pd.DataFrame.from_dict(dict_edge_count_con, orient='index')
        df_edge_count_con.index.name = 'name'
        df_edge_count_con.to_csv(f'{outpath_branching}/conditional_branching_{variable_name}_{variable_val}.csv')

In [ ]:
# Identify records with temporal deviations
TEMPORAL_DEVIATION = 3  # z-score

dict_edge_duration = {}
for edge_from in list_activities:
    for edge_to in list_activities:
        dict_edge_duration[(edge_from, edge_to)] = []

for case_time, case_act in zip(df_ehrs['timestamp'], df_ehrs['activity_name']):
    assert len(case_time) == len(case_act)

    for idx in range(1, len(case_time)):
        edge_name = (case_act[idx-1], case_act[idx])
        edge_duration = (case_time[idx] - case_time[idx-1]).total_seconds() / (60*60)

        dict_edge_duration[edge_name].append(edge_duration)

dict_edge_stats = {}
for edge_name, list_edge_durations in dict_edge_duration.items():
    if len(list_edge_durations) != 0:
        dict_edge_stats[edge_name] = {'mean': np.mean(list_edge_durations),
                                      'std': np.std(list_edge_durations)
                                      }

        if np.std(list_edge_durations) == 0:
            dict_edge_stats[edge_name]['std'] = 1e-10  # To prevent divide by 0

    else:
        dict_edge_stats[edge_name] = {'mean': 0.,
                                      'std': 1e-10
                                      }

list_deviation_count = []
for case_time, case_act in zip(df_ehrs['timestamp'], df_ehrs['activity_name']):
    assert len(case_time) == len(case_act)

    if TEMPORAL_DEVIATION != 0:
        case_deviation_count = 0
        for idx in range(1,len(case_time)):
            edge_name = (case_act[idx-1], case_act[idx])
            edge_duration = (case_time[idx] - case_time[idx-1]).total_seconds() / (60*60)

            if abs((edge_duration - dict_edge_stats[edge_name]['mean']) / dict_edge_stats[edge_name]['std']) >= TEMPORAL_DEVIATION:
                case_deviation_count += 1

        list_deviation_count.append(case_deviation_count)
    else:
        list_deviation_count.append(0)

df_ehrs.loc[:, 'temporal_deviation_count'] = list_deviation_count

In [ ]:
# Save execution time distributions for prospective and retrospective cohort types
outpath_time = f'params/time_std{TEMPORAL_DEVIATION}'
if not os.path.exists(outpath_time):
    os.makedirs(outpath_time)

for variable_name in tqdm(['groupname', 'acuity']):
    for variable_val in tqdm(set(df_ehrs[variable_name].values)):
        df_patientgroup = df_ehrs[df_ehrs[variable_name] == variable_val]

        df_patientgroup = df_patientgroup[df_patientgroup['temporal_deviation_count'] == 0]

        dict_edge_duration = {}
        for edge_from in list_activities:
            for edge_to in list_activities:
                dict_edge_duration[(edge_from, edge_to)] = []

        for case_time, case_act in zip(df_patientgroup['timestamp'], df_patientgroup['activity_name']):
            assert len(case_time) == len(case_act)

            for idx in range(1, len(case_time)):
                edge_name = (case_act[idx-1], case_act[idx])
                edge_duration = (case_time[idx] - case_time[idx-1]).total_seconds() / (60*60)

                dict_edge_duration[edge_name].append(edge_duration)

        list_edge_names = []
        list_edge_time_distribution = []
        for edge_name, list_edge_durations in dict_edge_duration.items():
            list_edge_names.append(edge_name[0] + edge_name[1])

            if len(list_edge_durations) != 0:
                dict_duration_count = {}
                all_count = 0
                for duration_idx in list_edge_durations:
                    if duration_idx not in dict_duration_count.keys():
                        dict_duration_count[duration_idx] = 1
                    else:
                        dict_duration_count[duration_idx] += 1
                    all_count += 1

                dict_duration_count_relative = {key: value/all_count for key, value in dict_duration_count.items()}
                list_edge_time_distribution.append(dict_duration_count_relative)

            else:
                list_edge_time_distribution.append(0)

        df_pairwise = pd.DataFrame({'edge': list_edge_names, 'param': list_edge_time_distribution})
        df_pairwise.to_csv(f'{outpath_time}/time_{variable_name}_{variable_val}.csv', index=False)